# Capstone Project - Battle Of Neighborhoods

## This is the capstone project for the IBM Data Science Professional Certificate
### Open new Mexican Restaurant in London, United Kingdom

### 1. Introduction/Business Problem:

In [273]:
# 1. A description of the problem and a discussion of the background

London is the capital and largest city of England and the United Kingdom. It is the third-most populous in Europe. London has a diverse range of people and cultures, and more than 300 languages are spoken in the region.  It offers many business opportunities, like openning new restaurants so is competitive. Careful consideration of analysis for the business expansion is so much important as it is directly proportional to the cost of the business. The analysis from this report help for the new businesses strategically target the market and help in a high return on investment so is low risk. 

Opening a new restaurant can be daunting. There are so many things to consider from choosing the right location to finding the financing to selecting the right name. As the location is one of the main factors in opening a new restaurant we will focus on the restaurant business around the London area in this report. The first step in opening a new restaurant is deciding what type of restaurant it is going to be? Because that's already decided and it's going to be a specific type of cuisine i.e Mexican restaurant, now choosing the location of the restaurant is one of the most imortant decisions. Will it be in a busy area with plenty of foot traffic, or is there enough parking or are there any other restaurants opened or closed in the same spot?
We will focus on the neighborhoods that has no or least Mexican restaurants, also will take into consideration (with help of clustering) which neighbourhoods are similar and most like Mexican food but at the sam time has no or very little Mexican restaurants. Then I will hopefully come to a conclusion where it is a good place to open a new Mexican restaurant.

In [274]:
# 2. A description of the data and how it will be used to solve the problem

#### Target Audience:

The objective of my project is to recommend which neighborhood of London is a good choice for a new Mexican Restaurant business to open.

#### Methodology - How will I use the data to solve the described Business Problem:

- I will convert address data into their equivalent latitude and longitude values.
- For London neighborhood data, I will use the resource from wikipedia page , https://en.wikipedia.org/wiki/List_of_areas_of_London
- Use the Foursquare API to explore London neighborhoods and to get venues in neighborhoods.
- Use the Foursquare API to get venue ratings and likes in neighborhoods.
- Use the k-means clustering to complete clustering task
- Use the Folium library to visualize the neighborhoods, venues , clusters in London

### 2. Data section

 Download and Explore Borough Dataset

In [275]:
#install libraries 
!pip install geopy    
!pip install folium   
!pip install geocoder
print('Libraries installed')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

In [276]:
#import libraries 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt # for graphical usage 

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap

from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


#### Getting the latitude and longitude of London, United Kingdom

In [277]:
#get the coordinates of London
london_address = 'London, United Kingdom'

geolocator = Nominatim(user_agent='london_explorer')
location = geolocator.geocode(london_address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordiantes of {} are {}, {}'.format(london_address, latitude, longitude))

The geographical coordiantes of London, United Kingdom are 51.5073219, -0.1276474


##### Getting the boroughs data of London

I will be using the wikipedia page https://en.wikipedia.org/wiki/List_of_London_boroughs to get the data for 32 London boroughs and the City of London. The City of London is the 33rd principal division of Greater London but it is not a London borough, so it's not in the table that I'll be using, but I will manualy add the City of London as the 33th borough so that I can do a more extensive analysis.

In [278]:
# Read London borough data 
url = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
source=requests.get(url).text
soup=BeautifulSoup(source, 'html.parser')

In [279]:
# Get the whole table that we need from the wikipedia page and put it in a dataframe format
table = soup.find('table', class_="wikitable sortable")

df = pd.read_html(str(table))
df = pd.concat(df)

In [280]:
df

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham[note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20
5,Camden,NaN,NaN,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,270029,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W,11
6,Croydon,NaN,NaN,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,386710,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W,19
7,Ealing,NaN,NaN,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,341806,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W,13
8,Enfield,NaN,NaN,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,333794,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W,30
9,Greenwich [note 2],[note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,287942,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E,22


In [281]:
# Data cleansing, extract only the latitude and longitude (last 18 charachters) from the Co-ordinates column
i=0
for i in range(32):
    df['Co-ordinates'][i]= df['Co-ordinates'][i][-18:]
    i=i+1

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [282]:
# Droping the columns that i don't need
df=df.drop(columns=['Inner', 'Status', 'Local authority', 'Political control', 'Headquarters', 'Area (sq mi)', 'Population (2019 est)[1]', 'Nr. in map'])

In [283]:
# Check the dataframe
df

,Borough,Co-ordinates
0,Barking and Dagenham[note 1],51.5607°N 0.1557°E
1,Barnet,51.6252°N 0.1517°W
2,Bexley,51.4549°N 0.1505°E
3,Brent,51.5588°N 0.2817°W
4,Bromley,51.4039°N 0.0198°E
5,Camden,51.5290°N 0.1255°W
6,Croydon,51.3714°N 0.0977°W
7,Ealing,51.5130°N 0.3089°W
8,Enfield,51.6538°N 0.0799°W
9,Greenwich [note 2],51.4892°N 0.0648°E


In [284]:
# Add two columns for latitude and longitude, so that I can divide the coordiantes from 'Co-ordinates' column
df["Latitude"] = ""
df['Longitude']= ""

In [285]:
df

,Borough,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham[note 1],51.5607°N 0.1557°E,,
1,Barnet,51.6252°N 0.1517°W,,
2,Bexley,51.4549°N 0.1505°E,,
3,Brent,51.5588°N 0.2817°W,,
4,Bromley,51.4039°N 0.0198°E,,
5,Camden,51.5290°N 0.1255°W,,
6,Croydon,51.3714°N 0.0977°W,,
7,Ealing,51.5130°N 0.3089°W,,
8,Enfield,51.6538°N 0.0799°W,,
9,Greenwich [note 2],51.4892°N 0.0648°E,,


In [286]:
# Fill the two new columns with the latitude and longitude from 'Co-ordinates columns, take only the numbers (without nonnumerich characters)
import re
k=0
for k in range(32):
    df['Latitude'][k]=df['Co-ordinates'][k].split()[0][:7]
    df['Longitude'][k]=df['Co-ordinates'][k].split()[1][:6]
    k=k+1

In [287]:
#Check the dataframe
df

,Borough,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham[note 1],51.5607°N 0.1557°E,51.5607,0.1557
1,Barnet,51.6252°N 0.1517°W,51.6252,0.1517
2,Bexley,51.4549°N 0.1505°E,51.4549,0.1505
3,Brent,51.5588°N 0.2817°W,51.5588,0.2817
4,Bromley,51.4039°N 0.0198°E,51.4039,0.0198
5,Camden,51.5290°N 0.1255°W,51.5290,0.1255
6,Croydon,51.3714°N 0.0977°W,51.3714,0.0977
7,Ealing,51.5130°N 0.3089°W,51.5130,0.3089
8,Enfield,51.6538°N 0.0799°W,51.6538,0.0799
9,Greenwich [note 2],51.4892°N 0.0648°E,51.4892,0.0648


In [288]:
# Drop the columns 'Co-ordinates' because I don't need it anymore
df.drop('Co-ordinates', 1, inplace=True)

In [289]:
# As I mention on the start City of London is not is the table because is't not consoderd a borough, but I add it manually so that I can take in into consideration when I'll we doing the data analysis later
df.append({'Borough':'City of London', 'Latitude': '51.5155', 'Longitude': 0.0922}, ignore_index=True)

,Borough,Latitude,Longitude
0,Barking and Dagenham[note 1],51.5607,0.1557
1,Barnet,51.6252,0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,0.2817
4,Bromley,51.4039,0.0198
5,Camden,51.5290,0.1255
6,Croydon,51.3714,0.0977
7,Ealing,51.5130,0.3089
8,Enfield,51.6538,0.0799
9,Greenwich [note 2],51.4892,0.0648
